In [1]:
from taxcalc import *
import pandas as pd
import behresp as br

In [2]:
ref_0={}
ref_1={
    "II_brk5": {2021: [211844, 400000, 211844, 211844, 400000],
                2026: [502101, 502101, 251050, 502101, 502101]},
    "II_rt6": {2021: 0.33,
               2026: 0.35},
    "II_brk6": {2021: [416700, 416700, 317775, 416700, 416700],
               2026: [504149, 567168, 283575, 535659, 567168]},
    "II_rt7": {2021: 0.35,
               2026: 0.396},
    "II_brk7": {2021: [418400, 470700, 400000, 444550, 470700],
                2026: [9e99, 9e99, 9e99, 9e99, 9e99]},
    "II_rt8": {2021: 0.396,
               2026: 1},
    "PT_brk5": {2021: [211844, 400000, 211844, 211844, 400000],
                2026: [502101, 502101, 251050, 502101, 502101]},
    "PT_rt6": {2021: 0.33,
               2026: 0.35},
    "PT_brk6": {2021: [416700, 416700, 317775, 416700, 416700],
               2026: [504149, 567168, 283575, 535659, 567168]},
    "PT_rt7": {2021: 0.35,
               2026: 0.396},
    "PT_brk7": {2021: [418400, 470700, 400000, 444550, 470700],
                2026: [9e99, 9e99, 9e99, 9e99, 9e99]},
    "PT_rt8": {2021: 0.396,
               2026: 1}
}
ref_2={
    "ID_prt": {2021: 0.03},
    "ID_ps": {2021: [400000, 400000, 400000, 400000, 400000]}
}
ref_3={
    "CG_rt4": {2021: 0.396},
    "CG_brk3": {2021: [1000000, 1000000, 1000000, 1000000, 1000000]}
}
ref_4={"EITC_MaxEligAge": {2021: 125}}
ref_5={
    "ID_BenefitCap_rt": {2021: 0.28,
                         2026: 1},
    "ID_BenefitCap_Switch": {2021: [True, True, False, True, True, True, True],
                             2026: [True, True, True, True, True, True, True]}
}
ref_6={"SS_Earnings_thd": {2021: 400000}}

In [3]:
refs_biden = [ref_1, ref_2, ref_3, ref_4, ref_5, ref_6]
refs_base = [ref_0, ref_1, ref_2, ref_3, ref_4, ref_5]
rec = Records()

df = pd.DataFrame(index=["Restore pre-TCJA rates above $400,000 of income",
                         "Restore limitation on itemized deductions above $400,000 of income",
                         "Tax capital gains and dividends at same rate as ordinary income above $1 million of income",
                         "Extend the EITC to childless workers age 65 and older",
                         "Limit tax benefit of itemized deductions to 28 percent of value",
                         "Apply 12.4 percent OASDI payroll tax to eranings above $400,000"])
for year in range(2021, 2031):
    pol_base = Policy()
    pol_ref = Policy()
    col=[]
    # NOTE: the assumed capital gains response elasticity has a large effect on revenue.
    # TPC's response elasticity is adjusted for taxing "unrealized capital
    # gains at death, which reduces the incentive to defer realizations of gains."
    # Even though Tax-Calculator is not able to simulate this aspect of Biden's
    # proposal, we use TPC's response elasticity for purposes of comparison. 
    response_elasticities = {'sub': 0, 'inc': 0, 'cg': -2}
    # "stack" reform provisions in the same order as TPC
    for (ref_biden, ref_base) in zip(refs_biden, refs_base): 
        pol_base.implement_reform(ref_base)
        calc_base=Calculator(pol_base, rec)
        calc_base.advance_to_year(year)
        
        pol_ref.implement_reform(ref_biden)
        calc_ref=Calculator(pol_ref, rec)
        calc_ref.advance_to_year(year)
        
        base_df, ref_df = br.response(calc_base, calc_ref, response_elasticities)
        base_rev= (base_df['combined'] * base_df['s006']).sum()/1e9
        ref_rev = (ref_df['combined'] * ref_df['s006']).sum()/1e9
        
        dif_rev = ref_rev - base_rev
        col.append(round(dif_rev,2))
    df[year] = col
        

In [22]:
df_final = df.append(df.sum().rename('Total'))
print("The Tax-Brain simulation can be found at: https://compute.studio/PSLmodels/Tax-Brain/46299/")
df_final

The Tax-Brain simulation can be found at: https://compute.studio/PSLmodels/Tax-Brain/46299/


,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030
"Restore pre-TCJA rates above $400,000 of income",33.58,34.99,36.53,38.15,39.72,0.00,0.00,0.00,0.00,0.00
"Restore limitation on itemized deductions above $400,000 of income",11.75,12.19,12.68,13.19,13.76,-1.31,-1.36,-1.42,-1.49,-1.52
Tax capital gains and dividends at same rate as ordinary income above $1 million of income,10.82,11.23,10.90,11.29,11.22,5.81,5.81,6.10,6.37,6.81
Extend the EITC to childless workers age 65 and older,-0.51,-0.54,-0.56,-0.58,-0.61,-0.63,-0.66,-0.67,-0.69,-0.70
Limit tax benefit of itemized deductions to 28 percent of value,1.32,1.39,1.44,1.49,1.55,0.00,0.00,0.00,0.00,0.00
"Apply 12.4 percent OASDI payroll tax to eranings above $400,000",83.10,88.65,94.25,100.02,106.08,112.55,119.84,127.71,135.82,144.05
Total,140.06,147.91,155.24,163.56,171.72,116.42,123.63,131.72,140.01,148.64


In [18]:
tpc_data = {"Restore pre-TCJA rates above $400,000 of income": [17.4, 26.3, 28.2, 30.1, 31.7, 9.6, 0, 0, 0, 0],
            "Restore limitation on itemized deductions above $400,000 of income": [7.6, 13.1, 13.7, 14.4, 15.0, 6.1, 0, 0, 0, 0],
            "Tax capital gains and dividends at same rate as ordinary income above $1 million of income and tax unrealized capital gains at death": [5.3, 34.2, 47.4, 49.4, 52.0, 48.9, 48.2, 51.0, 54.0, 57.2],
            "Extend the EITC to childless workers age 65 and older": [0.0, -0.4, -0.4, -0.4, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5],
            "Limit tax benefit of itemized deductions to 28 percent of value": [8.6, 14.9, 15.5, 16.2, 16.8, 33.6, 46.2, 48.8, 51.4, 54.2],
            "Apply 12.4 percent OASDI payroll tax to eranings above $400,000": [51.9, 74.4, 80.6, 87.1, 92.8, 99.9, 107.2, 114.9, 122.6, 131.1]}

tpc_df = pd.DataFrame.from_dict(tpc_data, orient='index')
tpc_df.columns=range(2021,2031)
print("TPC revenue estimates by fiscal year")
print("NOTE: TPC's capital gains reform revenue estimates include taxing unrealized capital gains at death")
print("NOTE: TPC's methodology for limiting itemized deductions to 28 percent of value differ from DeBacker, Jensen, and Metz's methodology")
print("Source: https://www.taxpolicycenter.org/sites/default/files/publication/158624/An_Analysis_of_Former_Vice_President_Bidens_Tax_Proposals_1_0.pdf")
tpc_df


TPC revenue estimates by fiscal year
NOTE: TPC's capital gains reform revenue estimates include taxing unrealized capital gains at death
NOTE: TPC's methodology for limiting itemized deductions to 28 percent of value differ from DeBacker, Jensen, and Metz's methodology
Source: https://www.taxpolicycenter.org/sites/default/files/publication/158624/An_Analysis_of_Former_Vice_President_Bidens_Tax_Proposals_1_0.pdf


,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030
"Restore pre-TCJA rates above $400,000 of income",17.4,26.3,28.2,30.1,31.7,9.6,0.0,0.0,0.0,0.0
"Restore limitation on itemized deductions above $400,000 of income",7.6,13.1,13.7,14.4,15.0,6.1,0.0,0.0,0.0,0.0
Tax capital gains and dividends at same rate as ordinary income above $1 million of income and tax unrealized capital gains at death,5.3,34.2,47.4,49.4,52.0,48.9,48.2,51.0,54.0,57.2
Extend the EITC to childless workers age 65 and older,0.0,-0.4,-0.4,-0.4,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5
Limit tax benefit of itemized deductions to 28 percent of value,8.6,14.9,15.5,16.2,16.8,33.6,46.2,48.8,51.4,54.2
"Apply 12.4 percent OASDI payroll tax to eranings above $400,000",51.9,74.4,80.6,87.1,92.8,99.9,107.2,114.9,122.6,131.1
